# Abundance Estimation with seqs.qza

After dereplication we are left with a single file "mags_derep_all_domains.qza". This file is then subject to abundance estimation with the already existing file "updog_seqs.qza".


### Workflow

Once we recover MAGs from metagenomic data, we may be interested in estimating their abundance in the samples. We can do it by mapping the original reads (updog_seqs.qza) to the dereplicated MAGs (mags_derep_all_domains.qza) and calculating the abundance based on the read mapping results. There are a couple of ways to estimate MAG abundance, such as RPKM (Reads Per Kilobase per Million mapped reads) and TPM (Transcripts Per Million). Here we will use TPM to estimate the abundance of each MAG in all samples.

### 1. Get MAG Lentgth

This step calculates the lengths of each dereplicated MAG, which will be used in the next step to estimate abundance.

In [ ]:
mosh annotate get-feature-lengths \
    --i-features ./mags_derep_all_domains.qza \              
    --o-lengths ./mags_derep_all_domains_length.qza \ #not sure if .qza necessary here 

### 2. Index dereplicated MAGs

This step indexes the dereplicated MAGs for read mapping. The index is necessary to efficiently map the input reads back to the MAGs.

In [ ]:
mosh assembly index-derep-mags \
    --i-mags ./mags_derep_all_domains.qza \                  
    --p-threads 8 \  
    --p-seed 100 \    #check parameter settings                               
    --o-index ./mags_derep_all_domains_index.qza \                            

### 3. Map reads to dereplicated MAGs

In this step, we map the input paired-end reads back to the dereplicated MAGs. This helps in calculating the abundance of each MAG in the sample.

In [ ]:
mosh assembly map-reads \
    --i-index ./mags_derep_all_domains_index.qza \                            
    --i-reads ./reads_filtered.qza \   #not sure wich file 
    --p-threads 8 \  
    --p-seed 100 \                  
    --o-alignment-maps ./reads_to_derep_mags.qza \

### 4. Estimate MAG abundance

This step estimates the abundance of each MAG in the sample based on the read mapping results.

metric : TPM (Transcripts Per Million)

min-mapq : indicates the minimum required read mapping quality — for Bowtie2, 42 will allow only perfect matches to be retained

min-base-quality : only keep alignments with this minimal Phred quality score

In [ ]:
mosh annotate estimate-abundance \
    --i-feature-lengths ./mags_derep_all_domains_length.qza \
    --i-alignment-maps ./reads_to_derep_mags.qza \
    --p-threads 10 \
    --p-metric tpm \
    --p-min-mapq 42 \
    --o-abundances ./mags_derep_ft.qza \

### 5. Looking at estimated MAG Abundance

In [ ]:
mosh annotate classify-kraken2 \
    --i-seqs ./mags_derep_all_domains.qza \
    --i-db ./kraken2_db \
    --p-threads 40 \
    --p-confidence 0.5 \
    --p-report-minimizer-data \
    --o-reports ./kraken_reports_mags_derep.qua \
    --o-outputs ./kraken_hits_mags_derep.qza \

Then we will convert a Kraken 2 report into a generic taxonomy artifact for downstream analyses.

In [ ]:
mosh annotate kraken2-to-mag-features \
    --i-reports ./kraken_reports_mags_derep.qza  \
    --i-outputs ./kraken_hits_mags_derep.qza  \
    --o-taxonomy ./mags_derep_taxonomy.qza \

Now we are ready to generate a taxa bar plot.

In [ ]:
mosh taxa barplot \
    --i-table ./mags_derep_ft.qza \ #generated in Step 4.
    --i-taxonomy ./mags_derep_taxonomy.qza \
    --m-metadata-file ./cocoa-metadata.tsv \
    --o-visualization ./results/mags-derep-taxa-bar-plot.qzv \

In [ ]:
Visualization.load(f"{data_dir}/mags-derep-taxa-bar-plot.qzv")